In [13]:
import pandas as pd
import os
import json
from tqdm import tqdm

In [15]:
POSTS_OF_EACH_RATING = 210_000
tag_ratings_json = []

rating_map = {
    'q': {
        'done': False,
        'posts': []
    },
    's': {
        'done': False,
        'posts': []
    },
    'e': {
        'done': False,
        'posts': []
    }
}


done_reading_files = False

def clean_tag_string(tag_string: str) -> str:
    bad_chars_map = {
        '_': ' ',
        '(': '',
        ')': '',
        '-': ' ',
        '!': ''
    }

    cleaned_tag_string = tag_string.strip().lower()

    for old_char, new_char in bad_chars_map.items():
        cleaned_tag_string = cleaned_tag_string.replace(old_char, new_char)
    

    return " ".join(set(cleaned_tag_string.split(" ")))


for datafile in tqdm(os.listdir('data')):
    with open(os.path.join('data', datafile), 'r') as file:
        while file:
            try:
                line = file.readline()
                if line == "": break

                post_data = json.loads(line)
                
                if not rating_map[post_data['rating']]['done']:
                    rating_map[post_data['rating']]['posts'].append({
                        'rating': post_data['rating'],
                        'tags': clean_tag_string(post_data['tag_string'])
                    })
                
                if len(rating_map[post_data['rating']]['posts']) >= POSTS_OF_EACH_RATING:
                    rating_map[post_data['rating']]['done'] = True

                if all(metadata['done'] for metadata in rating_map.values()):
                    done_reading_files = True
                    break

            except:
                pass

    if done_reading_files:
        break
    
for metadata in rating_map.values():
    tag_ratings_json += metadata['posts']

tag_ratings_json[:10]

 78%|███████▊  | 7/9 [00:45<00:13,  6.52s/it]


[{'rating': 'q',
  'tags': 'online midriff long hair girls multiple ribbon thief 2girls hunter morii ragnarok shizuki'},
 {'rating': 'q',
  'tags': 'panty breasts rabbit uniform underwear pull panties red large skirt kanon transistor kawasumi solo baby hata 1girl school mai ears animal'},
 {'rating': 'q',
  'tags': 'konata swimsuit star ryouou uniform bowtie lucky one naka no under long solo serafuku 1girl school bow izumi pants clothes yellow hair yumeno piece'},
 {'rating': 'q',
  'tags': 'sensei yukihito kasuga red eyes long aisaka hitodama hair solo negima white ass 1girl mahou sayo kneeling'},
 {'rating': 'q',
  'tags': '3d hyung blonde long etaine hair girls shaiya multiple persona kim 2girls jun black dual'},
 {'rating': 'q',
  'tags': 'ryu3224 solo ooyama gouketsuji gloves 1girl thighhighs ichizoku kicking rin'},
 {'rating': 'q',
  'tags': 'eyes swimsuit trail recorder back black support wet one dripping leaning original arm glasses solo 1girl school kneeling instrument short c

In [16]:
tags_df = pd.DataFrame.from_records(tag_ratings_json).dropna().drop_duplicates(subset=['tags'])
tags_df.head(n=10)

,rating,tags
0,q,online midriff long hair girls multiple ribbon...
1,q,panty breasts rabbit uniform underwear pull pa...
2,q,konata swimsuit star ryouou uniform bowtie luc...
3,q,sensei yukihito kasuga red eyes long aisaka hi...
4,q,3d hyung blonde long etaine hair girls shaiya ...
5,q,ryu3224 solo ooyama gouketsuji gloves 1girl th...
6,q,eyes swimsuit trail recorder back black suppor...
7,q,panties blush blue eyes panty grabbing hair gr...
8,q,inuyasha breasts eyeshadow medium weapon black...
9,q,hinoue little busters girls hair multiple saig...


In [17]:
# tags_df.shape

tags_df.groupby(by='rating').count()

,tags
rating,
e,208437
q,209293
s,209722


In [8]:
tags_df.to_csv(os.path.join('data', 'danbooru_tags.csv'), index=False)